<div>
    <img style="float:right;" src="images/smi-logo.png"/>
    <div style="float:left;color:#58288C;"><h1>Relational Databases and Data Warehousing</h1></div>
</div>

---
# Notebook 3: Intro to Data Warehousing
In this notebook you learn to turn an existing transactional database into a simple Data Warehouse.

Requirements:
- You have completed the previous exercise on relational databases
- You have a database file ready that is structured according to your data model and contains mock data for customers, products and transactions.
- You are familiar with the key principles of Data Warehousing.
- You are familiar with the concepts of fact and dimension tables in star schemas.
- You have already created a star schema model and are ready to implement it in an actual database.
---

Let's load our existing database.

In [1]:
%load_ext sql
%sql sqlite:///data/my-database.db

---
## <span style="color:#FF5D02;">Assigment: Create star schema</span>
To simplify the exercise, we continue working with the existing database. Your task is to add the tables for the dimensions first, then add the required fact tables for the transactional data.

Name the dimensions ``dim_<dimension-name>``, e.g. dim_customer, and the fact tables ``fact_<fact-name>``, e.g. fact_orders.

> __NOTE:__ Before you start you should create a backup copy of the database with the results from Notebook 2 in case you somehow damage the data while experimenting.

### Create dimension tables
Let's start with a first dimension that probably every Data Warehouse contains - time. We pick one possible design out of many here, we define weekly granularity as the lowest one:

In [ ]:
%%sql

-- create time dimension

DROP TABLE IF EXISTS dim_time;
CREATE TABLE dim_time (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    week INTEGER NOT NULL,
    month INTEGER NOT NULL,
    year INTEGER NOT NULL
);

Please add tables for the dimensions from your star schema model. Be sure to include the state (i.e. New York, California, ...) in which a customer lives as an attribute of customers.

In [ ]:
%%sql

-- create customers dimension
DROP TABLE IF EXISTS dim_customers;
CREATE TABLE dim_customers (
    ...
);

-- create ... dimension
...

### Create fact tables
Please add the fact tables from your star schema model.

In [ ]:
%%sql

-- create orders fact table
DROP TABLE IF EXISTS fact_orders;
CREATE TABLE fact_orders (
    ...
);

...


---
## <span style="color:#FF5D02;">Assigment: Populate Data Warehouse</span>
To simulate an ETL process, please use the ``INSERT INTO table SELECT ...`` statement to copy the transactional data:
- Use your newly created tables from Notebook #2 as data source (or the mock_* tables if you started this notebook from a fresh environment). 
- Copy the data to your newly created star schema tables dim_* and fact_*.

### Load dimension data

Let's start with populating the time dimension. To to this we have to create a list of all points in time that are relevant to the data warehouse: weeks, months and years:
- the ``strftime`` function extracts which week, month, year it is per date from the order table
- the ``SELECT DISTICT`` eliminates duplicates and thus prevents the same date from being added multiple times.

Please adjust the statement so it fits your data structure.

In [ ]:
%%sql 

-- first: let's try that...

SELECT DISTINCT strftime("%W", order_date) as no_of_week, strftime("%m", order_date) as month, strftime("%Y", order_date) as year
FROM orders ORDER BY order_date;

In [ ]:
%%sql 

-- now ingest that data into the time dimension table

INSERT INTO dim_time (week, month, year) 
    SELECT DISTINCT strftime("%W", order_date), strftime("%m", order_date), strftime("%Y", order_date) 
    FROM orders ORDER BY order_date;

Now please ETL the dimensional data for all other dimensions.

> Hint: To populate the ``state`` field of customers you can use the expression ``substr(address, -8, 2)`` which extracts two characters at the position 8 chars before string end, where the state information sits in the mock dataset.

In [ ]:
%%sql 

INSERT INTO dim_customers (...) SELECT ... ;
...

### Load fact data
Now please ETL the fact data from your transactional data tables (hint: orders, reviews). Be sure to let each row in the fact table refer to all relevant dimensions be referring to a row in the dimensions tables by id.

Example:

- dim_customer has:
  ``id 42: John Doe``
  
- dim_time has:
  ``id 1: week 2, month 1, year 2022``
  
- fact_orders refers to these to dimensions and adds the attributes of that fact:
  ``cust_id: 42, time_id: 1, ..., quantitiy: 10``

If you feel unsure at this point, recheck with the literature specified in the online course.

>Hint: If you load facts that are associated with a point in time, you have match that date with the relevant points in time, our data warehouse is aware of (-> dim_time table dimension):
> - In the source table, you want to load data from, decompose the date into week, month, year (using ``strftime``) - this allows you to match the date to the time dimension.
> - Join the source table with dim_time on week, month, year to get the time id for each row in your source table.
> - Load the resulting data table into the fact table.

In [ ]:
%%sql 

-- load facts to fact_...

INSERT INTO fact_... (time_id, <dimension ids and fact-table columns>)
SELECT dim_time.id, <relevant fact columns from source table>
FROM <transactional table> source
JOIN dim_time t ON strftime("%Y", source.date_column) = dim_time.year 
    AND strftime("%m", source.date_column) = dim_time.month 
    AND strftime("%W", source.date_column) = dim_time.week;
JOIN dim_... ON ... -- potential other dimension
JOIN dim_... ON ... -- potential other dimension


Now you have created a tiny data warehouse. To query the dwh, you usually select from a fact table and connect all relevant dimensions to your query with ``JOIN``s. You can use this query as a template:

In [ ]:
%%sql

SELECT * FROM fact_... 
JOIN dim_customers c ON fact_...cust_id = c.id
JOIN dim_...

---
## <span style="color:#FF5D02;">Assigment: Query Data Warehouse</span>
Create to queries to the Data Warehouse to check if everything works as expected. 

### 1. Generate a statistic of # of purchases and average price for New York Citizens by dimension product category
First, we want to check, if the data is imported correctly. So we recreate tge same statistic from the data warehouse that you created from the order_log view at the end of Notebook #2. 

> Hint: Use the dimensions products and customers.

In [ ]:
%%sql


### 2. Generate a statistic of average reviews over states for category "Accessories"
With your result, try to answer this question: Is there a state that rates Accessories significantly worse than others? If so, which one?

In [ ]:
%%sql



If you want to continue working on Notebook #4 later in another session, please download your current state of my-database.db and upload it later when you come back.